In [1]:
import os
import random
import time
import json
import warnings 
warnings.filterwarnings('ignore')

import segmentation_models_pytorch as smp

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from utils import label_accuracy_score

import cv2

import numpy as np
import pandas as pd

# 전처리를 위한 라이브러리
from pycocotools.coco import COCO
import torchvision
import torchvision.transforms as transforms
import torchgeometry as tgm

import albumentations as A
from albumentations.pytorch import ToTensorV2

# 시각화를 위한 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from tqdm import tqdm

plt.rcParams['axes.grid'] = False

print('pytorch version: {}'.format(torch.__version__))
print('GPU 사용 가능 여부: {}'.format(torch.cuda.is_available()))

print(torch.cuda.get_device_name(0))
print(torch.cuda.device_count())


device = "cuda" if torch.cuda.is_available() else "cpu"   # GPU 사용 가능 여부에 따라 device 정보 저장

pytorch version: 1.4.0
GPU 사용 가능 여부: True
Tesla V100-PCIE-32GB
1


# 하이퍼파라미터, Seed 고정

In [2]:
# hyperparameter
class cfg:
    batch_size = 16
    num_epochs = 40
    learning_rate = 1e-4
    weight_decay = 1e-6
    model_name = "DeepLabV3Plus"
    encoders = "efficientnet-b4"
    encode_weights = "imagenet"

# data_path
dataset_path = '../input/data'
anns_file_path = dataset_path + '/' + 'train.json'

In [3]:
# albumentations.augmentations.transforms.HueSaturationValue (hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, always_apply=False, p=0.5)

In [4]:
# seed 고정
random_seed = 21
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
# torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.dete2rministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

In [5]:
category_names = ['Backgroud',
 'UNKNOWN',
 'General trash',
 'Paper',
 'Paper pack',
 'Metal',
 'Glass',
 'Plastic',
 'Styrofoam',
 'Plastic bag',
 'Battery',
 'Clothing']

# Loss 정의

In [6]:
class CustomLoss(nn.Module):
    def __init__(self):
        nn.Module.__init__(self)
        self.focal = tgm.losses.FocalLoss(alpha=0.5, gamma=2.0, reduction='mean')
        self.dice = tgm.losses.DiceLoss()

    def forward(self,input_tensor,target_tensor):
        focal_loss = self.focal(input_tensor,target_tensor)
        dice_loss = self.dice(input_tensor, target_tensor)
        
        loss = focal_loss + dice_loss
        return loss

# Dataset 정의

In [7]:
def get_classname(classID, cats):
    for i in range(len(cats)):
        if cats[i]['id']==classID:
            return cats[i]['name']
    return "None"

class CustomDataLoader(Dataset):
    """COCO format"""
    def __init__(self, data_dir, mode = 'train', transform = None):
        super().__init__()
        self.mode = mode
        self.transform = transform
        self.coco = COCO(data_dir)
        
    def __getitem__(self, index: int):
        # dataset이 index되어 list처럼 동작
        image_id = self.coco.getImgIds(imgIds=index)
        image_infos = self.coco.loadImgs(image_id)[0]
        
        # cv2 를 활용하여 image 불러오기
        images = cv2.imread(os.path.join(dataset_path, image_infos['file_name']))
        images = cv2.cvtColor(images, cv2.COLOR_BGR2RGB).astype(np.float32)
        images /= 255.0
        
        if (self.mode in ('train', 'val')):
            ann_ids = self.coco.getAnnIds(imgIds=image_infos['id'])
            anns = self.coco.loadAnns(ann_ids)

            # Load the categories in a variable
            cat_ids = self.coco.getCatIds()
            cats = self.coco.loadCats(cat_ids)

            # masks : size가 (height x width)인 2D
            # 각각의 pixel 값에는 "category id + 1" 할당
            # Background = 0
            masks = np.zeros((image_infos["height"], image_infos["width"]))
            # Unknown = 1, General trash = 2, ... , Cigarette = 11
            for i in range(len(anns)):
                className = get_classname(anns[i]['category_id'], cats)
                pixel_value = category_names.index(className)
                masks = np.maximum(self.coco.annToMask(anns[i])*pixel_value, masks)
            masks = masks.astype(np.float32)

            # transform -> albumentations 라이브러리 활용
            if self.transform is not None:
                transformed = self.transform(image=images, mask=masks)
                images = transformed["image"]
                masks = transformed["mask"]
            
            return images, masks, image_infos
        
        if self.mode == 'test':
            # transform -> albumentations 라이브러리 활용
            if self.transform is not None:
                transformed = self.transform(image=images)
                images = transformed["image"]
            
            return images, image_infos
    
    
    def __len__(self) -> int:
        # 전체 dataset의 size를 return
        return len(self.coco.getImgIds())

# Dataset, DataLoader

In [8]:
# train.json / validation.json / test.json 디렉토리 설정
train_path = dataset_path + '/train.json'
val_path = dataset_path + '/val.json'
test_path = dataset_path + '/test.json'

# collate_fn needs for batch
def collate_fn(batch):
    return tuple(zip(*batch))
#                             A.Resize(256,256),
#                             A.HorizontalFlip(p=1.0),
#                             A.CropNonEmptyMaskIfExists(256,256,p=0.5),
#                             A.RandomRotate90(p=0.5),
# #                             A.ElasticTransform(p=0.5),
#                             A.GridDistortion(p=0.5),
# #                             A.OpticalDistortion(p=0.5),
# #                             A.ShiftScaleRotate(p=0.5),
train_transform = A.Compose([
                            A.HorizontalFlip(p=0.5),
                            A.Rotate(limit=90),
                            A.transforms.ColorJitter(),
                            #A.OneOf([
                            #    A.ElasticTransform(p=0.5, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
                            #    A.GridDistortion(p=0.5),
                            #    A.OpticalDistortion(distort_limit=1, shift_limit=0.5, p=1),
                            #], p=0.5),
                            #A.Normalize(mean=[0.485, 0.456, 0.406],
                            #  std=[0.229, 0.224, 0.225]),
                            ToTensorV2()
                            
                            ])
train_dataset = CustomDataLoader(data_dir=train_path, mode='train', transform=train_transform)
val_transform = A.Compose([
                          A.HorizontalFlip(p=0.5),
                          A.Rotate(limit=90),
                          A.transforms.ColorJitter(),
                          #A.OneOf([
                          #      A.ElasticTransform(p=0.5, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
                          #      A.GridDistortion(p=0.5),
                          #      A.OpticalDistortion(distort_limit=1, shift_limit=0.5, p=1),
                          #  ], p=0.5),  
                          #A.Normalize(mean=[0.485, 0.456, 0.406],
                          #    std=[0.229, 0.224, 0.225]),
                          ToTensorV2()
                          ])

test_transform = A.Compose([
                            
                           ToTensorV2()
                           ])

# create own Dataset 1 (skip)
# validation set을 직접 나누고 싶은 경우
# random_split 사용하여 data set을 8:2 로 분할
# train_size = int(0.8*len(dataset))
# val_size = int(len(dataset)-train_size)
# dataset = CustomDataLoader(data_dir=train_path, mode='train', transform=transform)
# train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# create own Dataset 2
# train dataset
train_dataset = CustomDataLoader(data_dir=train_path, mode='train', transform=train_transform)

# validation dataset
val_dataset = CustomDataLoader(data_dir=val_path, mode='val', transform=val_transform)

# test dataset
test_dataset = CustomDataLoader(data_dir=test_path, mode='test', transform=test_transform)


# DataLoader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=cfg.batch_size,
                                           shuffle=True,
                                           num_workers=4,
                                           collate_fn=collate_fn)

val_loader = torch.utils.data.DataLoader(dataset=val_dataset, 
                                         batch_size=cfg.batch_size,
                                         shuffle=False,
                                         num_workers=4,
                                         collate_fn=collate_fn)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=16,
                                          num_workers=4,
                                          collate_fn=collate_fn)

loading annotations into memory...
Done (t=4.15s)
creating index...
index created!
loading annotations into memory...
Done (t=4.35s)
creating index...
index created!
loading annotations into memory...
Done (t=1.36s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [9]:
import math
from torch.optim import lr_scheduler
from torch.optim.lr_scheduler import _LRScheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim.lr_scheduler import CosineAnnealingLR
# from torch.optim.lr_scheduler import CosineAnnealingWarmUp

In [10]:

class CosineAnnealingWarmUpRestarts(_LRScheduler):
    def __init__(self, optimizer, T_0, T_mult=1, eta_max=0.1, T_up=0, gamma=1., last_epoch=-1):
        if T_0 <= 0 or not isinstance(T_0, int):
            raise ValueError("Expected positive integer T_0, but got {}".format(T_0))
        if T_mult < 1 or not isinstance(T_mult, int):
            raise ValueError("Expected integer T_mult >= 1, but got {}".format(T_mult))
        if T_up < 0 or not isinstance(T_up, int):
            raise ValueError("Expected positive integer T_up, but got {}".format(T_up))
        self.T_0 = T_0
        self.T_mult = T_mult
        self.base_eta_max = eta_max
        self.eta_max = eta_max
        self.T_up = T_up
        self.T_i = T_0
        self.gamma = gamma
        self.cycle = 0
        self.T_cur = last_epoch
        super(CosineAnnealingWarmUpRestarts, self).__init__(optimizer, last_epoch)
        
    
    def get_lr(self):
        if self.T_cur == -1:
            return self.base_lrs
        elif self.T_cur < self.T_up:
            return [(self.eta_max - base_lr)*self.T_cur / self.T_up + base_lr for base_lr in self.base_lrs]
        else:
            return [base_lr + (self.eta_max - base_lr) * (1 + math.cos(math.pi * (self.T_cur-self.T_up) / (self.T_i - self.T_up))) / 2
                    for base_lr in self.base_lrs]

    def step(self, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
            self.T_cur = self.T_cur + 1
            if self.T_cur >= self.T_i:
                self.cycle += 1
                self.T_cur = self.T_cur - self.T_i
                self.T_i = (self.T_i - self.T_up) * self.T_mult + self.T_up
        else:
            if epoch >= self.T_0:
                if self.T_mult == 1:
                    self.T_cur = epoch % self.T_0
                    self.cycle = epoch // self.T_0
                else:
                    n = int(math.log((epoch / self.T_0 * (self.T_mult - 1) + 1), self.T_mult))
                    self.cycle = n
                    self.T_cur = epoch - self.T_0 * (self.T_mult ** n - 1) / (self.T_mult - 1)
                    self.T_i = self.T_0 * self.T_mult ** (n)
            else:
                self.T_i = self.T_0
                self.T_cur = epoch
                
        self.eta_max = self.base_eta_max * (self.gamma**self.cycle)
        self.last_epoch = math.floor(epoch)
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

# train, validation

In [11]:
def train(num_epochs, model, data_loader, val_loader, criterion, optimizer, saved_dir, val_every, device):
    print('Start training..')
    best_loss = 9999999
    best_epoch = 0
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        cnt = 0
        for step, (images, masks, _) in enumerate(data_loader):
            images = torch.stack(images)       # (batch, channel, height, width)
            masks = torch.stack(masks).long()  # (batch, channel, height, width)
            
            # gpu 연산을 위해 device 할당
            images, masks = images.to(device), masks.to(device)
            
            # inference
            outputs = model(images)
            loss = criterion(outputs, masks)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()
            # step 주기에 따른 loss 출력
            if (step + 1) % 25 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(
                    epoch+1, num_epochs, step+1, len(train_loader), loss.item()))
                total_loss += loss.item()
                cnt += 1
        print(f'Train Average Loss:{(total_loss/cnt):.4f}')
        
        # validation 주기에 따른 loss 출력 및 best model 저장
        if (epoch + 1) % val_every == 0:
            avrg_loss = validation(epoch + 1, model, val_loader, criterion, device)
            if avrg_loss < best_loss:
                best_epoch = epoch + 1
                print('Best performance at epoch: {}'.format(epoch + 1))
                print('Save model in', saved_dir)
                best_loss = avrg_loss
                save_model(model, saved_dir)

In [12]:
def validation(epoch, model, data_loader, criterion, device):
    print('Start validation #{}'.format(epoch))
    model.eval()
    with torch.no_grad():
        total_loss = 0
        cnt = 0
        mIoU_list = []
        for step, (images, masks, _) in enumerate(data_loader):
            
            images = torch.stack(images)       # (batch, channel, height, width)
            masks = torch.stack(masks).long()  # (batch, channel, height, width)

            images, masks = images.to(device), masks.to(device)            

            outputs = model(images)
            loss = criterion(outputs, masks)
            total_loss += loss
            cnt += 1
            
            outputs = torch.argmax(outputs.squeeze(), dim=1).detach().cpu().numpy()

            mIoU = label_accuracy_score(masks.detach().cpu().numpy(), outputs, n_class=12)[2]
            mIoU_list.append(mIoU)
            
        avrg_loss = total_loss / cnt
        print('Validation #{}  Average Loss: {:.4f}, mIoU: {:.4f}'.format(epoch, avrg_loss, np.mean(mIoU_list)))

    return avrg_loss

# 모델 저장 함수 정의

In [13]:
# 모델 저장 함수 정의
val_every = 1 

saved_dir = './saved'
if not os.path.isdir(saved_dir):                                                           
    os.mkdir(saved_dir)
    
def save_model(model, saved_dir, file_name=f'{cfg.model_name}_{cfg.encoders}_{cfg.encode_weights}_best_model.pt'):
    check_point = {'net': model.state_dict()}
    output_path = os.path.join(saved_dir, file_name)
    torch.save(model.state_dict(), output_path)

# model, loss, optimizer 정의

In [14]:
!nvidia-smi

Tue May  4 19:52:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 418.56       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:00:05.0 Off |                  Off |
| N/A   39C    P0    26W / 250W |     11MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [15]:
!fuser -v /dev/nvidia*

                     USER        PID ACCESS COMMAND
/dev/nvidia-uvm:     root     kernel mount /dev/nvidia-uvm
                     root      19298 F.... python
/dev/nvidia-uvm-tools:
                     root     kernel mount /dev/nvidia-uvm-tools
/dev/nvidia0:        root     kernel mount /dev/nvidia0
                     root      19298 F...m python
/dev/nvidiactl:      root     kernel mount /dev/nvidiactl
                     root      19298 F.... python


In [16]:
if cfg.model_name in ["DeepLabV3Plus", "deeplabv3plus"]:
    model = smp.DeepLabV3Plus(encoder_name=cfg.encoders,
                              encoder_weights=cfg.encode_weights,
                              in_channels=3,
                              classes=12)
    
# elif cfg.model_name in ["DeepLabV3Plus","deeplabv3plus"]:
#     model = smp.DeepLabV3Plus(
#         encoder_name=cfg.encoders,        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
#         encoder_weights=cfg.encode_weights,     # use `imagenet` pre-trained weights for encoder initialization
#         in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
#         classes=12,  
#     )


x = torch.randn([2, 3, 512, 512])
print(f'model name: {cfg.model_name}')
print("input shape : ", x.shape)
out = model(x).to(device)
print("output shape : ", out.size())

model = model.to(device)

# Loss function 정의
criterion = nn.CrossEntropyLoss()
focal = tgm.losses.FocalLoss(alpha=0.5, gamma=2.0, reduction='mean')
dice = tgm.losses.DiceLoss()

custom = CustomLoss()
    
# Optimizer 정의
optimizer = torch.optim.Adam(params = model.parameters(), lr = cfg.learning_rate, weight_decay=cfg.weight_decay)
scheduler = CosineAnnealingWarmUpRestarts(optimizer,T_mult=1,eta_max=1e-4,T_0=20,T_up=2,gamma=0.5)


model name: DeepLabV3Plus
input shape :  torch.Size([2, 3, 512, 512])
output shape :  torch.Size([2, 12, 512, 512])


# Train Step

In [17]:
#### import time
start = time.time()
train(cfg.num_epochs, model, train_loader, val_loader, custom, optimizer, saved_dir, val_every, device)
print(f'\n\ntime:{time.time() - start}')

Start training..
Epoch [1/40], Step [25/164], Loss: 1.2824
Epoch [1/40], Step [50/164], Loss: 1.0776
Epoch [1/40], Step [75/164], Loss: 0.7593
Epoch [1/40], Step [100/164], Loss: 0.7442
Epoch [1/40], Step [125/164], Loss: 0.6196
Epoch [1/40], Step [150/164], Loss: 0.5938
Train Average Loss:0.8462
Start validation #1
Validation #1  Average Loss: 0.5728, mIoU: 0.2335
Best performance at epoch: 1
Save model in ./saved
Epoch [2/40], Step [25/164], Loss: 0.5758
Epoch [2/40], Step [50/164], Loss: 0.5069
Epoch [2/40], Step [75/164], Loss: 0.5049
Epoch [2/40], Step [100/164], Loss: 0.4307
Epoch [2/40], Step [125/164], Loss: 0.4447
Epoch [2/40], Step [150/164], Loss: 0.5341
Train Average Loss:0.4995
Start validation #2
Validation #2  Average Loss: 0.4106, mIoU: 0.2996
Best performance at epoch: 2
Save model in ./saved
Epoch [3/40], Step [25/164], Loss: 0.4676
Epoch [3/40], Step [50/164], Loss: 0.3666
Epoch [3/40], Step [75/164], Loss: 0.3395
Epoch [3/40], Step [100/164], Loss: 0.4139
Epoch [3/4

# load trained model

In [18]:
# best model 저장된 경로
model_path = f'./saved/{cfg.model_name}_{cfg.encoders}_{cfg.encode_weights}_best_model.pt'

# best model 불러오기
checkpoint = torch.load(model_path, map_location=device)
model.load_state_dict(checkpoint)

# 추론을 실행하기 전에는 반드시 설정 (batch normalization, dropout 를 평가 모드로 설정)
# model.eval()

<All keys matched successfully>

# test for submission

In [19]:
def test(model, data_loader, device):
    size = 256
    transform = A.Compose([A.Resize(256, 256)])
    print('Start prediction.')
    model.eval()
    
    file_name_list = []
    preds_array = np.empty((0, size*size), dtype=np.long)
    
    with torch.no_grad():
        for step, (imgs, image_infos) in enumerate(test_loader):

            # inference (512 x 512)
            outs = model(torch.stack(imgs).to(device))
            oms = torch.argmax(outs.squeeze(), dim=1).detach().cpu().numpy()
            
            # resize (256 x 256)
            temp_mask = []
            for img, mask in zip(np.stack(imgs), oms):
                transformed = transform(image=img, mask=mask)
                mask = transformed['mask']
                temp_mask.append(mask)

            oms = np.array(temp_mask)
            
            oms = oms.reshape([oms.shape[0], size*size]).astype(int)
            preds_array = np.vstack((preds_array, oms))
            
            file_name_list.append([i['file_name'] for i in image_infos])
    print("End prediction.")
    file_names = [y for x in file_name_list for y in x]
    
    return file_names, preds_array

# submission.csv 생성

In [20]:
# sample_submisson.csv 열기
submission = pd.read_csv('./submission/sample_submission.csv', index_col=None)

# test set에 대한 prediction
file_names, preds = test(model, test_loader, device)

# PredictionString 대입
for file_name, string in zip(file_names, preds):
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

# submission.csv로 저장
submission.to_csv(f"./submission/{cfg.model_name}_{cfg.encoders}_{cfg.encode_weights}_(pretrained).csv", index=False)

Start prediction.
End prediction.


# EDA

In [21]:
def get_classname(classID, cats):
    for i in range(len(cats)):
        if cats[i]['id']==classID:
            return cats[i]['name']
    return "None"

class CustomDataLoader2(Dataset):
    """COCO format"""
    def __init__(self, data_dir, mode = 'train', transform = None):
        super().__init__()
        self.mode = mode
        self.transform = transform
        self.coco = COCO(data_dir)
        
    def __getitem__(self, index: int):
        # dataset이 index되어 list처럼 동작
        image_id = self.coco.getImgIds(imgIds=index)
        image_infos = self.coco.loadImgs(image_id)[0]
        
        # cv2 를 활용하여 image 불러오기
        images = cv2.imread(os.path.join(dataset_path, image_infos['file_name']))
        images = cv2.cvtColor(images, cv2.COLOR_BGR2RGB).astype(np.float32)
        images /= 255.0
        images = images.astype(np.uint8)
        
        if (self.mode in ('train', 'val')):
            ann_ids = self.coco.getAnnIds(imgIds=image_infos['id'])
            anns = self.coco.loadAnns(ann_ids)

            # Load the categories in a variable
            cat_ids = self.coco.getCatIds()
            cats = self.coco.loadCats(cat_ids)

            # masks : size가 (height x width)인 2D
            # 각각의 pixel 값에는 "category id + 1" 할당
            # Background = 0
            masks = np.zeros((image_infos["height"], image_infos["width"]))
            # Unknown = 1, General trash = 2, ... , Cigarette = 11
            for i in range(len(anns)):
                className = get_classname(anns[i]['category_id'], cats)
                pixel_value = category_names.index(className)
                masks = np.maximum(self.coco.annToMask(anns[i])*pixel_value, masks)
            masks = masks.astype(np.uint8)
            
            # transform -> albumentations 라이브러리 활용
            if self.transform is not None:
                transformed = self.transform(image=images, mask=masks)
                images = transformed["image"]
                masks = transformed["mask"]
            
            return images, masks, image_infos
        
        if self.mode == 'test':
            # transform -> albumentations 라이브러리 활용
            if self.transform is not None:
                transformed = self.transform(image=images)
                images = transformed["image"]
            
            return images, image_infos
    
    
    def __len__(self) -> int:
        # 전체 dataset의 size를 return
        return len(self.coco.getImgIds())

In [22]:
def imshow_test_loader(test_loader, model):
    for imgs, masks in test_loader:
        temp_images = imgs
        
        model.eval()
        outs = model(torch.stack(temp_images).to(device))
        oms = torch.argmax(outs.squeeze(), dim=1).detach().cpu().numpy()
        break

    for idx, img in enumerate(temp_images):
        fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(12, 12))

        print('image shape:', list(temp_images[idx].shape))
        print('mask shape: ', list(oms[idx].shape))
        print('Unique values, category of transformed mask : \n', [{int(i),category_names[int(i)]} for i in list(np.unique(oms[idx]))])

        ax1.imshow(temp_images[idx].permute([1,2,0]))
        ax1.grid(False)

        ax2.imshow(oms[idx])
        ax2.grid(False)

        plt.show()

def imshow_train_loader(train_loader, model,w_cnt):
    cnt = 0  
    for imgs, masks, image_infos in train_loader:
        temp_images = imgs
        image_infos = image_infos[0]
        temp_masks = masks
    
        model.eval()
        outs = model(torch.stack(temp_images).to(device))
        oms = torch.argmax(outs.squeeze(), dim=1).detach().cpu().numpy()
        cnt+=1
        if w_cnt == cnt:
            break

    for idx, img in enumerate(temp_images):
        fig, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3, figsize=(24, 24))
    
        print('답안,Unique values, category of transformed mask : \n', [{int(i),category_names[int(i)]} for i in list(np.unique(temp_masks[idx]))])
        print('학습한거,Unique values, category of transformed mask : \n', [{int(i),category_names[int(i)]} for i in list(np.unique(oms[idx]))])

        ax1.imshow(temp_images[idx].permute([1,2,0]))
        ax1.grid(False)

        ax2.imshow(oms[idx])
        ax2.grid(False)

        ax3.imshow(temp_masks[idx])
        ax3.grid(False)
        
        plt.show()

In [23]:
torch.cuda.empty_cache()

In [24]:
'''
                            A.HorizontalFlip(p=0.5),
                            A.VerticalFlip(p=0.5),
                            A.RandomRotate90(),
                            A.transforms.ColorJitter(),
                            A.OneOf([
                                A.ElasticTransform(p=0.5, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
                                A.GridDistortion(p=0.5),
                                A.OpticalDistortion(distort_limit=1, shift_limit=0.5, p=1),
                            ], p=0.5),
                            A.Normalize(mean=[0.485, 0.456, 0.406],
                              std=[0.229, 0.224, 0.225]),
    '''

EDA_transform = A.Compose([
                            A.transforms.CLAHE(),
                            ])

EDA_dataset = CustomDataLoader2(data_dir=train_path, mode='train', transform=EDA_transform)

test_EDA_dataset = CustomDataLoader2(data_dir=test_path, mode='test', transform=EDA_transform)

EDA_loader = torch.utils.data.DataLoader(dataset=EDA_dataset, 
                                           batch_size=8,
                                           shuffle=True,
                                           num_workers=4,
                                           collate_fn=collate_fn)

test_EDA_loader = torch.utils.data.DataLoader(dataset=test_EDA_dataset, 
                                           batch_size=8,
                                           shuffle=False,
                                           num_workers=4,
                                           collate_fn=collate_fn)


loading annotations into memory...
Done (t=4.37s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [25]:
# imshow_train_loader(EDA_loader,model,30)